In [2]:
import pandas as pd
import numpy as np
from datetime import datetime


date = datetime.now().strftime('%d-%m-%Y')
# date = '21-05-2025'
date

'30-05-2025'

In [3]:
old = pd.read_csv(f'input/{date}/fleet-device-mapping-updated.csv', dtype=str)
amenx_update = pd.read_csv(f'input/{date}/prime-edge.csv', dtype=str)
# chalo_update = pd.read_csv(f'input/{date}/chalo-update_19-05-2025.csv', dtype=str)

# Convert column names to snake case and remove special characters
old.columns = old.columns.str.lower().str.replace(
    '[^a-zA-Z0-9]', '_', regex=True).str.replace('__+', '_', regex=True).str.strip('_').str.strip()
amenx_update.columns = amenx_update.columns.str.lower().str.replace(
    '[^a-zA-Z0-9]', '_', regex=True).str.replace('__+', '_', regex=True).str.strip('_').str.strip()
# chalo_update.columns = chalo_update.columns.str.lower().str.replace(
#     '[^a-zA-Z0-9]', '_', regex=True).str.replace('__+', '_', regex=True).str.strip('_').str.strip()

In [4]:
old.info()
amenx_update.info()
# chalo_update.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3386 entries, 0 to 3385
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dep             3386 non-null   object
 1   fleet           3386 non-null   object
 2   type            3384 non-null   object
 3   regndate        3386 non-null   object
 4   obu_iemi        3124 non-null   object
 5   chalo_deviceid  2840 non-null   object
 6   date            3386 non-null   object
 7   vehicle_number  2579 non-null   object
 8   depot           2579 non-null   object
 9   ny_device_id    8 non-null      object
dtypes: object(10)
memory usage: 264.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   sl_n                 10 non-null     object
 1   depot                10 non-null     object
 2   fleet               

In [5]:
# Count rows where both obu_iemi and chalo_deviceid are NA
na_df = old[(old['obu_iemi'].isna()) & (old['chalo_deviceid'].isna())]
na_count = na_df.shape[0]
print(f"Number of rows where both obu_iemi and chalo_deviceid are NA: {na_count}")
na_df


Number of rows where both obu_iemi and chalo_deviceid are NA: 0


,dep,fleet,type,regndate,obu_iemi,chalo_deviceid,date,vehicle_number,depot,ny_device_id


In [6]:
old_copy = old.copy()

na_count = old_copy[(old_copy['obu_iemi'].isna()) & (
    old_copy['chalo_deviceid'].isna())].shape[0]
print(
    f"Number of rows where both obu_iemi and chalo_deviceid are NA: {na_count}")

Number of rows where both obu_iemi and chalo_deviceid are NA: 0


In [7]:
# Merge the dataframes on 'fleet' column to align indices
amenx_merged_df = pd.merge(old_copy, amenx_update[['fleet', 'amnex_iemi_updated']], 
                    on='fleet', how='left')

# Update obu_iemi where amnex_iemi_updated exists
old_copy['obu_iemi'] = np.where(
      (old_copy['fleet'] == amenx_merged_df['fleet']) & 
      (amenx_merged_df['amnex_iemi_updated'].notna()), 
    amenx_merged_df['amnex_iemi_updated'], old_copy['obu_iemi'])

In [8]:
amenx_merged_df

,dep,fleet,type,regndate,obu_iemi,chalo_deviceid,date,vehicle_number,depot,ny_device_id,amnex_iemi_updated
0,AN,I0001,ORDI,2006-07-10,NaN,869244042749626,2025-04-24T11:57:08,NaN,NaN,NaN,NaN
1,AN,I0504,ORDI,2007-07-13,NaN,869244044664427,2025-04-17T11:24:49,NaN,NaN,NaN,NaN
2,AN,I0508,ORDI,2007-07-13,NaN,869244044490302,2025-04-24T11:57:08,NaN,NaN,NaN,NaN
3,AD,I0510,ORDI,2007-07-19,NaN,869244044492688,2025-04-17T11:24:49,NaN,NaN,NaN,NaN
4,CW,I0515,ORDI,2007-08-01,NaN,869244044500415,2025-04-17T11:24:49,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3381,AN,S0006,AC,2020-02-07,867383050933771,869244042625834,2025-04-17T11:24:49,TN45N4325,Annanagar Depot,NaN,NaN
3382,AB,S0007,MINI,2016-02-29,862607059149962,868728039287195,2025-04-17T11:24:49,TN01AN1375,Adambakkam Depot,NaN,NaN
3383,TR,S0008,MINI,2016-03-01,868922051347470,869244042750939,2025-04-17T11:24:49,TN01AN1401,Thiruvanmiyur Depot,NaN,NaN
3384,KB,S0009,AC,2019-11-05,867032053718917,NaN,2025-04-17T11:24:49,TN72N2226,Kilambakkam Depot,NaN,NaN


In [48]:
chalo_merged_df = pd.merge(old_copy, chalo_update[['fleet', 'chalo_iemi_updated']],
                           on='fleet', how='left')

old_copy['chalo_deviceid'] = np.where(
    (old_copy['fleet'] == chalo_merged_df['fleet']) &
    (chalo_merged_df['chalo_iemi_updated'].notna()),
  chalo_merged_df['chalo_iemi_updated'], old_copy['chalo_deviceid'])

In [9]:
na_count = old_copy[(old_copy['obu_iemi'].isna()) & (
    old_copy['chalo_deviceid'].isna())].shape[0]
print(
    f"Number of rows where both obu_iemi and chalo_deviceid are NA: {na_count}")

Number of rows where both obu_iemi and chalo_deviceid are NA: 0


In [10]:
old_copy.to_csv(f'output/{date}/fleet-device-mapping-{date}.csv', index=False)